In [1]:
import numpy as np
from IPython.display import set_matplotlib_formats
from PIL import Image, ExifTags
import random
import cv2 
import PIL
import os
import glob
import Augmentor
import pylab
from Augmentor.Operations import Operation

# Image Augmentation with Augmentor
 [link](https://augmentor.readthedocs.io/en/master/index.html) 

## change image orientation

Check if all images have the correct orientation, if not change them and overwrite the old image in the folder.

In [2]:
home = os.getcwd().split('AUT-CNN-TUB')[0]
labels = ['1','2','13','14','7.1','8','5','4','3','10','12','11','6','7.2','15','9']
for label in labels:
    aug_img_path =os.path.join(home,'/home/jeronimo/git/AUT-CNN-TUB/Data/Raw_Data_27072018_canon/', str(label))
    aug_img_path_list = os.listdir(aug_img_path)

    for i, img_name in enumerate(aug_img_path_list):
        try:
            image=Image.open(aug_img_path + '/' + img_name)
            for orientation in ExifTags.TAGS.keys():
                if ExifTags.TAGS[orientation]=='Orientation':
                    break
            exif=dict(image._getexif().items())

            if exif[orientation] == 3:
                image=image.rotate(180, expand=True)
            elif exif[orientation] == 6:
                image=image.rotate(270, expand=True)
            elif exif[orientation] == 8:
                image=image.rotate(90, expand=True)
            image.save(aug_img_path + '/' + img_name)
            image.close()

        except (AttributeError, KeyError, IndexError):
            # cases: image don't have getexif
            pass

## custom operations

It's possible to create custom operation. This is useful if you want change the implemented operation or build those how not implemented yet. Bearbeiten

Add __Salt__ and __Pepper__ and change the __Random-Erase__ area.

In [3]:
# Create your new operation by inheriting from the Operation superclass:
class SaltPepper(Operation):
    # Here you can accept as many custom parameters as required:
    def __init__(self, probability, salt_vs_pepper = 0.2, amount = 0.004):
        # Call the superclass's constructor (meaning you must
        # supply a probability value):
        Operation.__init__(self, probability)
        # Set your custom operation's member variables here as required:
        self.salt_vs_pepper = salt_vs_pepper
        self.amount = amount


    # Your class must implement the perform_operation method:
    def perform_operation(self, images):
        # Start of code to perform custom image operation.
        augmented_images = []
        for image in images:

            image_array = np.array(image)

            row, col, _ = image_array.shape    

            num_salt = np.ceil(self.amount * row * col  * self.salt_vs_pepper)
            num_pepper = np.ceil(self.amount * row * col * (1.0 - self.salt_vs_pepper))

            # Add Salt noise
            coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image_array.shape]
            image_array[coords[0], coords[1], :] = 0

            # Add Pepper noise
            coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image_array.shape]
            image_array[coords[0], coords[1], :] = 255

            # End of code to perform custom image operation.

            # Return the image so that it can further processed in the pipeline:
            augmented_images.append(Image.fromarray(image_array))
        return augmented_images
    
class RandomErasing(Operation):
    """
    Class that performs Random Erasing, an augmentation technique described
    in `https://arxiv.org/abs/1708.04896 <https://arxiv.org/abs/1708.04896>`_
    by Zhong et al. To quote the authors, random erasing:
    "*... randomly selects a rectangle region in an image, and erases its
    pixels with random values.*"
    Exactly this is provided by this class.
    Random Erasing can make a trained neural network more robust to occlusion.
    """
    def __init__(self, probability, rectangle_area):
        """
        The size of the random rectangle is controlled using the
        :attr:`rectangle_area` parameter. This area is random in its
        width and height.
        :param probability: The probability that the operation will be
         performed.
        :param rectangle_area: The percentage are of the image to occlude.
        """
        Operation.__init__(self, probability)
        self.rectangle_area = rectangle_area

    def perform_operation(self, images):
        """
        Adds a random noise rectangle to a random area of the passed image,
        returning the original image with this rectangle superimposed.
        :param images: The image(s) to add a random noise rectangle to.
        :type images: List containing PIL.Image object(s).
        :return: The transformed image(s) as a list of object(s) of type
         PIL.Image.
        """
        augmented_images = []
        for image in images:

            w, h = image.size

            w_occlusion_max = int(w * self.rectangle_area)
            h_occlusion_max = int(h * self.rectangle_area)

            w_occlusion_min = int(w * 0.01)
            h_occlusion_min = int(h * 0.01)

            w_occlusion = random.randint(w_occlusion_min, w_occlusion_max)
            h_occlusion = random.randint(h_occlusion_min, h_occlusion_max)

            if len(image.getbands()) == 1:
                rectangle = Image.fromarray(np.uint8(np.random.rand(w_occlusion, h_occlusion) * 255))
            else:
                rectangle = Image.fromarray(np.uint8(np.random.rand(w_occlusion, h_occlusion,
                                                                    len(image.getbands())) * 255))

            random_position_x = random.randint(10, w - (w_occlusion + 10))
            random_position_y = random.randint(10, h - (h_occlusion + 10))

            image.paste(rectangle, (random_position_x, random_position_y))
            augmented_images.append(image)

        return augmented_images
    

# augment canon images

In [6]:
###### Namen der Ordner 
labels = ['1','2','13','14','7.1','8','5','4','3','10','12','11','6','7.2','15','9']

# Dateipfat den den Ordnern mit den Bildern.
# D.h. im Ordner '.../Centered_28x28' die Ordner '1', '2' uns

# Definieren des Pfades zu den Bildern, die Bearbeitet werden sollen
home = os.getcwd().split('AUT-CNN-TUB')[0]
data_path  = os.path.join(home,'AUT-CNN-TUB/Data/Raw_Data_27072018_canon/')

# custom operations
salt_pepper = SaltPepper(probability=0.5 , salt_vs_pepper= 0.2, amount= 0.004)
random_erasing = RandomErasing(probability=0.8, rectangle_area=0.05)

# Es werden alle oben angegebnen Ordner durchlaufen
for label in labels:
    # Anzahl der bilder Pro Ornderer (Bauteilart) ermitteln
    img_path = os.path.join(data_path+label)
    num_img = len(os.listdir(img_path))
    
    # Augmentor-Objekt wird angelegt mit aktuellem Ordner
    p = Augmentor.Pipeline(data_path+label, 
                           output_directory=os.path.join(home,'AUT-CNN-TUB/Data/27072018/aug/canon',label ))
    
    # definieren der Operationen
    # Hier können gern noch weitere hinzugefügt werden oder Wahrscheinlichkeiten geändert werden. 
    p.flip_left_right(probability=0.2)
    p.rotate(probability=0.2, max_left_rotation=5, max_right_rotation=5)
    #p.random_erasing(probability=0.8, rectangle_area=0.11)
    #p.random_erasing(probability=0.8, rectangle_area=0.11)
    p.random_brightness(0.5, 1, 2)
    p.random_contrast(0.5, 1, 2)
    p.random_color(0.2, 0.5, 10)
    p.add_operation(salt_pepper)
    p.add_operation(random_erasing)
    p.add_operation(random_erasing)

    
    # n ist proportional zur Anzahl der Bilder im Orner
    # Aktuell werden nur 5% der Bilder gezogen. Später gerne mehr!
    n = int(np.round(3 * num_img))
    p.sample(n)
    
    aug_img_path = os.path.join(home,'AUT-CNN-TUB/Data/27072018/aug/canon',label )
    aug_img_path_list = os.listdir(aug_img_path)
        
    for i, img_name in enumerate(aug_img_path_list):
        if '_original_' in img_name:
            new_name = img_name.split(sep='_original_')[1][0:13] + '_aug_' + str(i) + '.jpg'

            old_file = os.path.join(aug_img_path, img_name)
            new_file = os.path.join(aug_img_path, new_name)

            os.rename(old_file, new_file)



Executing Pipeline:   0%|          | 0/612 [00:00<?, ? Samples/s]

Initialised with 204 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/canon/1.

Processing <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F0F91AA7048>: 100%|██████████| 612/612 [00:14<00:00, 41.51 Samples/s] 
Executing Pipeline:   0%|          | 0/564 [00:00<?, ? Samples/s]

Initialised with 188 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/canon/2.

Processing <PIL.Image.Image image mode=RGB size=720x480 at 0x7F0F91ABC2B0>: 100%|██████████| 564/564 [00:14<00:00, 38.05 Samples/s]                   
Executing Pipeline:   0%|          | 0/471 [00:00<?, ? Samples/s]

...

Processing <PIL.Image.Image image mode=RGB size=720x480 at 0x7F0F933C5B70>: 100%|██████████| 369/369 [00:08<00:00, 43.48 Samples/s]                  
Executing Pipeline:   0%|          | 0/492 [00:00<?, ? Samples/s]

Initialised with 164 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/canon/15.

Processing <PIL.Image.Image image mode=RGB size=720x480 at 0x7F0F91B173C8>: 100%|██████████| 492/492 [00:11<00:00, 42.01 Samples/s]                   
Executing Pipeline:   0%|          | 0/450 [00:00<?, ? Samples/s]

Initialised with 150 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/canon/9.

Processing <PIL.Image.Image image mode=RGB size=720x480 at 0x7F0FC8519208>: 100%|██████████| 450/450 [00:11<00:00, 40.90 Samples/s]                   


# augment industry images

In [4]:

###### Namen der Ordner 
labels = ['1','2','13','14','7.1','8','5','4','3','10','12','11','6','7.2','15','9']

# Dateipfat den den Ordnern mit den Bildern.
# D.h. im Ordner '.../Centered_28x28' die Ordner '1', '2' uns

# Definieren des Pfades zu den Bildern, die Bearbeitet werden sollen
home = os.getcwd().split('AUT-CNN-TUB')[0]
data_path  = os.path.join(home,'AUT-CNN-TUB/Data/Raw_Data_27072018_industry/')

# custom operations
salt_pepper = SaltPepper(probability=0.5 , salt_vs_pepper= 0.2, amount= 0.004)
random_erasing = RandomErasing(probability=0.9, rectangle_area=0.01)

# Es werden alle oben angegebnen Ordner durchlaufen
for label in labels:
    # Anzahl der bilder Pro Ornderer (Bauteilart) ermitteln
    img_path = os.path.join(data_path+label)
    num_img = len(os.listdir(img_path))
    
    # Augmentor-Objekt wird angelegt mit aktuellem Ordner
    p = Augmentor.Pipeline(data_path+label, 
                           output_directory=os.path.join(home,'AUT-CNN-TUB/Data/27072018/aug/industry',label ))
    
    # definieren der Operationen
    # Hier können gern noch weitere hinzugefügt werden oder Wahrscheinlichkeiten geändert werden. 
    p.flip_left_right(probability=0.2)
    p.rotate(probability=0.4, max_left_rotation=10, max_right_rotation=10)
    #p.random_erasing(probability=0.8, rectangle_area=0.11)
    #p.random_erasing(probability=0.8, rectangle_area=0.11)
    p.random_brightness(0.5, 0.7, 1.5)
    p.random_contrast(0.5, 0.5, 1.5)
    p.random_color(0.2, 0.5, 10)
    p.add_operation(salt_pepper)
    p.add_operation(random_erasing)
    p.add_operation(random_erasing)

    
    # n ist proportional zur Anzahl der Bilder im Orner
    # Aktuell werden nur 5% der Bilder gezogen. Später gerne mehr!
    n = int(np.round(1 * num_img))
    p.sample(n)
    
    aug_img_path = os.path.join(home,'AUT-CNN-TUB/Data/27072018/aug/industry',label )
    aug_img_path_list = os.listdir(aug_img_path)
        
    for i, img_name in enumerate(aug_img_path_list):
        new_name = img_name.split(sep='_original_')[1][0:13] + '_aug_' + str(i) + '.jpg'

        old_file = os.path.join(aug_img_path, img_name)
        new_file = os.path.join(aug_img_path, new_name)

        os.rename(old_file, new_file)

Executing Pipeline:   0%|          | 0/1160 [00:00<?, ? Samples/s]

Initialised with 1160 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/industry/1.

Processing <PIL.Image.Image image mode=RGB size=1280x1024 at 0x7FB861A1B978>: 100%|██████████| 1160/1160 [01:32<00:00, 12.48 Samples/s]                  
Executing Pipeline:   0%|          | 0/1097 [00:00<?, ? Samples/s]

Initialised with 1097 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/industry/2.

Processing <PIL.Image.Image image mode=RGB size=1280x1024 at 0x7FB8601992E8>: 100%|██████████| 1097/1097 [01:27<00:00, 12.50 Samples/s]                  
Executing Pipeline:   0%|          | 0/1075 [00:00<?, ? Samples/s]

...

Processing <PIL.Image.Image image mode=RGB size=1280x1024 at 0x7FB861A61400>: 100%|██████████| 1257/1257 [01:45<00:00, 11.95 Samples/s]                  
Executing Pipeline:   0%|          | 0/1059 [00:00<?, ? Samples/s]

Initialised with 1059 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/industry/15.

Processing <PIL.Image.Image image mode=RGB size=1280x1024 at 0x7FB8602B4A20>: 100%|██████████| 1059/1059 [01:30<00:00, 11.76 Samples/s]                 
Executing Pipeline:   0%|          | 0/948 [00:00<?, ? Samples/s]

Initialised with 948 image(s) found.
Output directory set to /home/jeronimo/git/AUT-CNN-TUB/Data/27072018/aug/industry/9.

Processing <PIL.Image.Image image mode=RGB size=1280x1024 at 0x7FB8602683C8>: 100%|██████████| 948/948 [01:24<00:00, 11.19 Samples/s]                  
